In [ ]:
# %pip install imageio-ffmpeg
# %pip install dm-reverb[tensorflow]
# %pip install atari_py
# %pip install imageio
# %pip install PILLOW
# %pip install tf-agents[reverb]
# %pip install gin-config
# %pip install pyvirtualdisplay
# %pip install typing-extensions
# %pip install matplotlib
# %pip install pybullet
# %pip install tensorboard

In [ ]:
from __future__ import absolute_import, division, print_function

import os
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import tempfile
import reverb

import tensorflow as tf

from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

from nnpugail import NnPugailAgent, CategoricalCritic, CategoricalActor, EnvCritic
from tf_agents.networks import actor_distribution_network
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import tanh_normal_projection_network

print(tf.version.VERSION)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3048)])
    except RuntimeError as e:
        print(e)

Dedicating a separate logs dir for the example will help us group different examples in tensorboard:

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}
model_name = "nn-PUGAIL"
tempdir = os.path.join("logs", model_name, env_name.split('-')[0])
# tempdir = tempfile.gettempdir()

# Setup

## Hyperparameters

In [ ]:
# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 1000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 1000 # @param {type:"integer"}

batch_size = 16 # @param {type:"integer"}

memory_len = 5 # @param {type: "integer"}
n_turns = 3 # @param {type: "integer"}
num_atoms = 51 # @param {type "integer"}
q_value = 20 # @param {type "integer"}

disc_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
etha_learning_rate = 3e-4 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (32, 16)
critic_fc_layer_params = (32, 16)
disc_joint_fc_layer_params = (32, 16)

log_interval = 500 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 1000 # @param {type:"integer"}

policy_save_interval = 1000 # @param {type:"integer"}

## Environment

Environments in RL represent the task or problem that we are trying to solve. Standard environments can be easily created in TF-Agents using suites. We have different suites for loading environments from sources such as the OpenAI Gym, Atari, DM Control, etc., given a string environment name.

Now let's load the Minituar environment from the Pybullet suite.

In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

In this environment the goal is for the agent to train a policy that will control the Minitaur robot and have it move forward as fast as possible. Episodes last 1000 steps and the return will be the sum of rewards throughout the episode. Observation and action specs of this environment are following:

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Reward Spec:')
print(env.time_step_spec().reward)
print('Action Spec:')
print(env.action_spec())

The observation is fairly complex. We receive 28 values representing the angles, velocities, and torques for all the motors. In return the environment expects 8 values for the actions between [-1, 1]. These are the desired motor angles.

In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

## Distribution Strategy

In [ ]:
use_gpu = False

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

## Agent

The descriminator will give us value estimates for Q(s,a). That is, it will recieve as input an observation, an action and a reward, and it will give us an estimate of how good that action was for the given state.

In [ ]:
from tf_agents.specs import tensor_spec

observation_spec, action_spec, time_step_spec = (
    spec_utils.get_tensor_specs(collect_env))
reward_spec = tensor_spec.BoundedTensorSpec(
        shape=[], dtype=tf.float32, minimum=tf.float32.min, maximum=tf.float32.max)
with strategy.scope():
    disc_net = critic_network.CriticNetwork(
    # disc_net = CategoricalCritic(
          (observation_spec, action_spec),
          observation_fc_layer_params=critic_fc_layer_params,
          action_fc_layer_params=critic_fc_layer_params,
        #   num_atoms=num_atoms,
        #   q_value=q_value,
          joint_fc_layer_params=disc_joint_fc_layer_params,
          kernel_initializer='glorot_normal',
          last_kernel_initializer='glorot_normal')

In [ ]:
with strategy.scope():
    rew_net = EnvCritic(
          observation_spec,
          observation_fc_layer_params=critic_fc_layer_params,
          action_fc_layer_params=critic_fc_layer_params,
          joint_fc_layer_params=disc_joint_fc_layer_params,
          kernel_initializer='glorot_normal',
          last_kernel_initializer='glorot_normal')


We will use this discriminator evaluate an actor network performance during training which will allow us to generate actions given an observation.

# Context Generator Demo

The ActorNetwork will predict parameters for a tanh-squashed MultivariateNormalDiag distribution. This distribution will then be sampled, conditioned on the current observation, whenever we need to generate actions.

In [ ]:
with strategy.scope():
    # actor_net = actor_distribution_network.ActorDistributionNetwork(
    actor_net = CategoricalActor(
        observation_spec,
        action_spec,
        fc_layer_params=actor_fc_layer_params,
        continuous_projection_net=(
            tanh_normal_projection_network.TanhNormalProjectionNetwork))

With these networks at hand we can now instantiate the agent.

In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = NnPugailAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        discriminator_network=disc_net,
        reward_network=rew_net,
        actor_optimizer=tf.keras.optimizers.Adam(
            learning_rate=actor_learning_rate),
        discriminator_optimizer=tf.keras.optimizers.Adam(
            learning_rate=disc_learning_rate),
        reward_model_optimizer=tf.keras.optimizers.Adam(
            learning_rate=disc_learning_rate),
        train_step_counter=train_step)

  tf_agent.initialize()

## Replay buffer

In [ ]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

In [ ]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## Policies

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
    tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
    tf_collect_policy, use_tf_function=True)

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
    collect_env.time_step_spec(), collect_env.action_spec())

## Actors

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
    reverb_replay.py_client,
    table_name,
    sequence_length=2,
    stride_length=1)

In [ ]:
initial_collect_actor = actor.Actor(
    collect_env,
    random_policy,
    train_step,
    steps_per_run=initial_collect_steps,
    observers=[rb_observer])
initial_collect_actor.run()

In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
    collect_env,
    collect_policy,
    train_step,
    steps_per_run=1,
    metrics=actor.collect_metrics(10),
    summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
    observers=[rb_observer, env_step_metric])

In [ ]:
eval_actor = actor.Actor(
    eval_env,
    eval_policy,
    train_step,
    episodes_per_run=num_eval_episodes,
    metrics=actor.eval_metrics(num_eval_episodes),
    summary_dir=os.path.join(tempdir, 'eval'),
)

## Learners

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)
# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
    tempdir,
    train_step,
    tf_agent,
    experience_dataset_fn,
    triggers=learning_triggers,
    strategy=strategy)

## Metrics and Evaluation

In [ ]:
def get_eval_metrics():
    eval_actor.run()
    results = {}
    for metric in eval_actor.metrics:
       results[metric.name] = metric.result()
    return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
    eval_results = (', ').join(
        '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
    print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

## Training the agent

In [ ]:
try:
    %timeit
except:
    pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
    # Training.
    collect_actor.run()
    loss_info = agent_learner.run(iterations=1)

    # Evaluating.
    step = agent_learner.train_step_numpy

    if eval_interval and step % eval_interval == 0:
        metrics = get_eval_metrics()
        log_eval_metrics(step, metrics)
        returns.append(metrics["AverageReturn"])

    if log_interval and step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## Visualization

In [ ]:
steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

In [ ]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

In [ ]:
num_episodes = 3
video_filename = f'{model_name}_{env_name.split("-")[0]}.mp4'
video_path = os.path.join('..', 'videos', video_filename)
with imageio.get_writer(video_path, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_path)

In [ ]:
!tensorboard --logdir {tempdir}

In [ ]:
tempdir